In [2]:
import os
import requests
import zipfile
import xml.etree.ElementTree as ET
import re

In [ ]:
import re

LINES_FILE = "movie_lines.txt"
CONVERSATIONS_FILE = "movie_conversations.txt"
OUTPUT_FILE = "cleaned_movie_dialogues.txt"

def load_lines(file_path):
    """ Läser in repliker och skapar en ordbok {LID: text} """
    line_dict = {}
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 5:
                line_id, text = parts[0], parts[4]
                line_dict[line_id] = text
    return line_dict

def extract_conversations(lines_dict, file_path, output_file):
    """ Läser in `movie_conversations.txt`, ersätter LIDs med repliker och sparar ren text """
    with open(file_path, "r", encoding="utf-8", errors="ignore") as f, open(output_file, "w", encoding="utf-8") as out:
        for line in f:
            parts = line.strip().split(" +++$+++ ")
            if len(parts) == 4:
                line_ids = eval(parts[3]) 
                conversation = " ".join(lines_dict.get(lid, "") for lid in line_ids)
                cleaned_text = re.sub(r"[^A-Za-z0-9.,!?;:\'\" ]+", " ", conversation) 
                out.write(cleaned_text.lower() + "\n") 

    print(f"Saved in {output_file}")


lines_dict = load_lines(LINES_FILE)
extract_conversations(lines_dict, CONVERSATIONS_FILE, OUTPUT_FILE)


✅ Sparat rensade dialoger i cleaned_movie_dialogues.txt


In [ ]:
import os
import re

GUTENBERG_DIR = "gutenberg_books"
CLEANED_GUTENBERG_FILE = "cleaned_gutenberg.txt"
MOVIE_DIALOGUES_FILE = "cleaned_movie_dialogues.txt"
FINAL_OUTPUT_FILE = "final_corpus.txt"

def clean_text(text):
    """ Tar bort Gutenberg-metadata, specialtecken och konverterar till lowercase. """
    text = re.sub(r"\*\*\* START OF THIS PROJECT GUTENBERG.*?\*\*\*", "", text, flags=re.DOTALL)
    text = re.sub(r"\*\*\* END OF THIS PROJECT GUTENBERG.*?\*\*\*", "", text, flags=re.DOTALL)
    text = re.sub(r"[^A-Za-z0-9.,!?;:\'\" ]+", " ", text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    return text.lower()

def process_gutenberg_books():
    """ Läser in och rensar alla böcker i `gutenberg_books/`, sparar dem i `cleaned_gutenberg.txt`. """
    all_texts = []

    for filename in os.listdir(GUTENBERG_DIR):
        if filename.endswith(".txt"):
            file_path = os.path.join(GUTENBERG_DIR, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                raw_text = f.read()
                cleaned_text = clean_text(raw_text)
                all_texts.append(cleaned_text)

    with open(CLEANED_GUTENBERG_FILE, "w", encoding="utf-8") as f:
        f.write("\n".join(all_texts))

    print(f"✅ Sparat rensad Gutenberg-text i {CLEANED_GUTENBERG_FILE}")

process_gutenberg_books()

✅ Sparat rensad Gutenberg-text i cleaned_gutenberg.txt


In [ ]:
def merge_cleaned_files():
    with open(CLEANED_GUTENBERG_FILE, "r", encoding="utf-8") as gutenberg, \
         open(MOVIE_DIALOGUES_FILE, "r", encoding="utf-8") as movies, \
         open(FINAL_OUTPUT_FILE, "w", encoding="utf-8") as final_file:
        

        final_file.write(gutenberg.read() + "\n")

        final_file.write(movies.read() + "\n")

    print(f"✅ Sparat slutlig dataset i {FINAL_OUTPUT_FILE}")

merge_cleaned_files()

✅ Sparat slutlig dataset i final_corpus.txt
